---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    with open("university_towns.txt") as f:
        townslist = f.readlines()
    
    townslist = [x.rstrip() for x in townslist]
    
    townslist2 = list()
    
    for i in townslist:
        if i[-6:] == '[edit]':
            temp_state = i[:-6]
        elif '(' in i:
            town = i[:i.index('(') - 1]
            townslist2.append([temp_state, town])
        else:
            town = i
            townslist2.append([temp_state, town])

    collegedf = pd.DataFrame(townslist2, columns=['State','RegionName'])


    return townslist
get_list_of_university_towns()

['Alabama[edit]',
 'Auburn (Auburn University)[1]',
 'Florence (University of North Alabama)',
 'Jacksonville (Jacksonville State University)[2]',
 'Livingston (University of West Alabama)[2]',
 'Montevallo (University of Montevallo)[2]',
 'Troy (Troy University)[2]',
 'Tuscaloosa (University of Alabama, Stillman College, Shelton State)[3][4]',
 'Tuskegee (Tuskegee University)[5]',
 'Alaska[edit]',
 'Fairbanks (University of Alaska Fairbanks)[2]',
 'Arizona[edit]',
 'Flagstaff (Northern Arizona University)[6]',
 'Tempe (Arizona State University)',
 'Tucson (University of Arizona)',
 'Arkansas[edit]',
 'Arkadelphia (Henderson State University, Ouachita Baptist University)[2]',
 'Conway (Central Baptist College, Hendrix College, University of Central Arkansas)[2]',
 'Fayetteville (University of Arkansas)[7]',
 'Jonesboro (Arkansas State University)[8]',
 'Magnolia (Southern Arkansas University)[2]',
 'Monticello (University of Arkansas at Monticello)[2]',
 'Russellville (Arkansas Tech Un

In [63]:
def get_recession_start():
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    
    gdp = pd.read_excel('gdplev.xls', skiprows = 207)
    gdp = gdp[['1996q4', 10739.1]]
    gdp.columns = ['quarter', 'gdp']
    for i in range(len(gdp['gdp'])):
        if i+2 == len(gdp['gdp']) :
            break
        elif (gdp.iloc[i+2, 1] < gdp.iloc[i+1, 1]) and (gdp.iloc[i+1, 1] < gdp.iloc[i, 1]):
            start = gdp.iloc[i-1, 0]
    

    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    return start
get_recession_start()

'2008q3'

In [67]:
def get_recession_end():
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    gdp = pd.read_excel('gdplev.xls', skiprows = 254)
    gdp = gdp[['2008q3', 14891.6]]
    gdp.columns = ['quarter', 'gdp']
    print(gdp)
    for i in range(len(gdp['gdp'])):
        if i+2 == len(gdp['gdp']) :
            break
        elif (gdp.iloc[i+2, 1] > gdp.iloc[i+1, 1]) and (gdp.iloc[i+1, 1] > gdp.iloc[i, 1]):
            end = gdp.iloc[i+2, 0]
            break
       
    return end
get_recession_end()

   quarter      gdp
0   2008q4  14577.0
1   2009q1  14375.0
2   2009q2  14355.6
3   2009q3  14402.5
4   2009q4  14541.9
5   2010q1  14604.8
6   2010q2  14745.9
7   2010q3  14845.5
8   2010q4  14939.0
9   2011q1  14881.3
10  2011q2  14989.6
11  2011q3  15021.1
12  2011q4  15190.3
13  2012q1  15291.0
14  2012q2  15362.4
15  2012q3  15380.8
16  2012q4  15384.3
17  2013q1  15491.9
18  2013q2  15521.6
19  2013q3  15641.3
20  2013q4  15793.9
21  2014q1  15747.0
22  2014q2  15900.8
23  2014q3  16094.5
24  2014q4  16186.7
25  2015q1  16269.0
26  2015q2  16374.2
27  2015q3  16454.9
28  2015q4  16490.7
29  2016q1  16525.0
30  2016q2  16583.1


'2009q4'

In [65]:
def get_recession_bottom():
    import pandas as pd
    pd.set_option('display.max_rows', 500)
    gdp = pd.read_excel('gdplev.xls')
    gdp = gdp[253:259]
    gdp = gdp[['Unnamed: 4', 'Unnamed: 6']]
    gdp.columns = ['quarter', 'gdp']
    bottom = 20000
    for i in range(len(gdp)):
        if gdp.iloc[i,1] < bottom:
            bottom = gdp.iloc[i,1]
            quarter = gdp.iloc[i,0]

    return quarter
get_recession_bottom()

'2009q2'

In [66]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    pd.set_option('display.max_columns', 100)
    df= pd.read_csv('City_Zhvi_AllHomes.csv')
    df['State'].replace(states, inplace = True)
    df = df.set_index(['State', 'RegionName'])
    for col in df.columns:
        if col.endswith('-01') or col.endswith('-02') or col.endswith('-03'):
            df = df.rename(columns = {col : col[:4]+'q1'})
        elif col.endswith('-04') or col.endswith('-05') or col.endswith('-06'):
            df = df.rename(columns = {col : col[:4]+'q2'})
        elif col.endswith('-07') or col.endswith('-08') or col.endswith('-09'):
            df = df.rename(columns = {col : col[:4]+'q3'})
        elif col.endswith('-10') or col.endswith('-11') or col.endswith('-12'):
            df = df.rename(columns = {col : col[:4]+'q4'})
    df = df.iloc[:,49:250]
    df = df.groupby(level=0, axis=1).mean()

    return df
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,2002q3,2002q4,2003q1,2003q2,2003q3,2003q4,2004q1,2004q2,2004q3,2004q4,2005q1,2005q2,2005q3,2005q4,2006q1,2006q2,2006q3,2006q4,2007q1,2007q2,2007q3,2007q4,2008q1,2008q2,2008q3,2008q4,2009q1,2009q2,2009q3,2009q4,2010q1,2010q2,2010q3,2010q4,2011q1,2011q2,2011q3,2011q4,2012q1,2012q2,2012q3,2012q4,2013q1,2013q2,2013q3,2013q4,2014q1,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.356500e+05,4.321333e+05,4.226667e+05,4.318000e+05,4.387667e+05,4.645000e+05,4.881333e+05,5.137000e+05,5.200333e+05,5.247000e+05,5.172333e+05,5.146667e+05,5.102000e+05,5.133667e+05,5.085000e+05,5.039333e+05,4.997667e+05,4.879333e+05,4.777333e+05,4.658333e+05,4.559333e+05,4.583667e+05,4.697000e+05,4.769333e+05,4.785667e+05,4.772667e+05,4.767000e+05,4.721333e+05,4.692000e+05,4.637000e+05,4.602000e+05,4.627000e+05,4.672333e+05,4.691667e+05,4.690333e+05,4.722333e+05,4.813667e+05,4.924333e+05,5.031333e+05,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
California,Los Angeles,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,2.870000e+05,3.022333e+05,3.189000e+05,3.345000e+05,3.506333e+05,3.708333e+05,3.952333e+05,4.255667e+05,4.561667e+05,4.773667e+05,4.950667e+05,5.184000e+05,5.455667e+05,5.710667e+05,5.817667e+05,5.875000e+05,5.929667e+05,5.925333e+05,5.917667e+05,5.895000e+05,5.773000e+05,5.613667e+05,5.353000e+05,5.022667e+05,4.695000e+05,4.439667e+05,4.262667e+05,4.139000e+05,4.063667e+05,4.043333e+05,4.133667e+05,4.111667e+05,4.067667e+05,4.020667e+05,3.971000e+05,3.901000e+05,3.821000e+05,3.755667e+05,3.706000e+05,3.753333e+05,3.855667e+05,3.965667e+05,4.115333e+05,4.328667e+05,4.550667e+05,4.719333e+05,4.842667e+05,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Illinois,Chicago,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,1.789667e+05,1.837000e+05,1.871000e+05,1.943000e+05,2.020667e+05,2.041333e+05,2.062000e+05,2.086000e+05,2.127667e+05,2.183333e+05,2.239000e+05,2.285333e+05,2.331667e+05,2.375000e+05,2.424000e+05,2.452000e+05,2.457000e+05,2.467000e+05,2.472333e+05,2.464000e+05,2.462667e+05,2.468333e+05,2.437333e+05,2.379000e+05,2.320000e+05,2.270333e+05,2.237667e+05,2.197000e+05,2.141000e+05,2.116667e+05,2.073000e+05,2.038000e+05,1.989333e+05,1.913667e+05,1.874000e+05,1.816000e+05,1.742667e+05,1.702000e+05,1.669333e+05,1.670333e+05,1.681333e+05,1.672333e+05,1.683667e+05,1.726333e+05,1.808000e+05,1.858667e+05,1.909000e+05,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Pennsylvania,Philadelphia,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,6.216667e+04,6.350000e+04,6.473333e+04,6.620000e+04,6.846667e+04,7.073333e+04,7.373333e+04,7.673333e+04,8.056667e+04,8.553333e+04,8.970000e+04,9.486667e+04,1.001667e+05,1.049000e+05,1.083333e+05,1.123667e+05,1.154667e+05,1.166333e+05,1.173333e+05,1.187000e+05,1.208333e+05,1.208000e+05,1.195667e+05,1.181333e+05,1.169333e+05,1.158667e+05,1.162000e+05,1.161667e+05,1.167333e+05,1.185667e+05,1.211667e+05,1.228000e+05,1.201667e+05,1.177667e+05,1.171000e+05,1.163000e+05,1.154667e+05,1.147000e+05,1.127333e+05,1.105333e+05,1.099333e+05,1.108333e+05,1.120000e+05,1.117333e+05,1.110000e+05,1.124667e+05,1.137333e+05,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Arizona,Phoenix,1.118333e+05,1.143667e+05,1.160000e+05,

In [69]:
def run_ttest():
    
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. '''
    
    new_df = convert_housing_data_to_quarters().loc[:, '2008q3':'2009q4']
    new_df['difference'] = new_df['2008q3'] - new_df['2009q4']
    new_df = new_df[['2008q3', '2009q4', 'difference']]
    
    university_town = pd.merge(get_list_of_university_towns(), new_df, how = 'inner', left_on = ['State', 'RegionName'], right_index=True)
    university_town = university_town.set_index(['State', 'RegionName'])
    
    non_university_town = pd.merge(university_town, new_df, how = 'outer', left_index=True, right_index=True)
    non_university_town = non_university_town[['2008q3_y', '2009q4_y', 'difference_y']]
    
    
    
    '''Then runs a ttest comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. '''
    
    #results = ttest_ind(university_town['difference'].dropna(), non_university_town['difference_y'].dropna())
    
    '''Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    t, p = ttest_ind(university_town['difference'].dropna(), non_university_town['difference_y'].dropna())
    different = True if p < 0.01 else False
    betters = "university town" if university_town['difference'].mean() < non_university_town['difference_y'].mean() else "non-university town"
    
    return t, p

run_ttest()

(-3.322984216458869, 0.00089375211484302663)